<a href="https://www.kaggle.com/code/darrentanzhijie/lstm-sentimental-analysis?scriptVersionId=214338665" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
import torch
from torch import nn
pd.set_option('display.max_colwidth', None)

In [2]:
df=pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

### Simple EDA

In [3]:
df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zom

In [4]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,"Loved today's show!!! It was a variety and not solely cooking (which would have been great too). Very stimulating and captivating, always keeping the viewer peeking around the corner to see what was coming up next. She is as down to earth and as personable as you get, like one of us which made the show all the more enjoyable. Special guests, who are friends as well made for a nice surprise too. Loved the 'first' theme and that the audience was invited to play along too. I must admit I was shocked to see her come in under her time limits on a few things, but she did it and by golly I'll be writing those recipes down. Saving time in the kitchen means more time with family. Those who haven't tuned in yet, find out what channel and the time, I assure you that you won't be disappointed.",positive
freq,5,25000


In [5]:
len(df)

50000

In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [8]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

df['sentiment']=le.fit_transform(df['sentiment'])

In [9]:
df['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [10]:
#Have to remove urls/tags, and all things non alpha numeric
str=df.iloc[1,0]
print(str)

A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.


In [11]:
import re
str=re.sub('<[^>]+>', ' ', str)
str=re.sub(r'https?://\S+', ' ', str)
str=re.sub('[^a-zA-Z0-9]',' ',str)
str

'A wonderful little production    The filming technique is very unassuming  very old time BBC fashion and gives a comforting  and sometimes discomforting  sense of realism to the entire piece    The actors are extremely well chosen  Michael Sheen not only  has got all the polari  but he has all the voices down pat too  You can truly see the seamless editing guided by the references to Williams  diary entries  not only is it well worth the watching but it is a terrificly written and performed piece  A masterful production about one of the great master s of comedy and his life    The realism really comes home with the little things  the fantasy of the guard which  rather than use the traditional  dream  techniques remains solid then disappears  It plays on our knowledge and our senses  particularly with the scenes concerning Orton and Halliwell and the sets  particularly of their flat with Halliwell s murals decorating every surface  are terribly well done '

### Data preprocessing

In [12]:
def cleaningtext(text):
    pattern1='<[^>]+>' #tags
    pattern2='https?://\S+' #urls
    pattern3='[^a-zA-Z0-9]' #non alpha numeric
    text=re.sub(pattern1,' ',text)
    text=re.sub(pattern2,' ',text)
    text=re.sub(pattern3,' ',text)
    return text.lower()

In [13]:
df['review']=df['review'].apply(cleaningtext)

In [14]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [15]:
#removing stop words and appyling Lemmatization

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
 
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()
def removing_stopwords(text):
    new=[]
    for w in word_tokenize(text):
        if w not in stop_words:
            new.append(lemmatizer.lemmatize(w))
    return ' '.join(new)



df['review']=df['review'].apply(removing_stopwords)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!




###  Spelling Correction Works

Detect Misspelled Words

Identify words in the text that are not valid according to a dictionary or linguistic model.

Suggest Corrections

Provide the closest valid word(s) based on the identified error.

Replace the Word

Replace the misspelled word with the most suitable correction.


In [16]:
pip install symspellpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 11.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
import symspellpy
from symspellpy import SymSpell, Verbosity

# Initialize SymSpell and load the dictionary
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load the dictionary (SymSpell default dictionary)
sym_spell.load_dictionary("/kaggle/input/frequency-dictionary-en-82-765-txt/frequency_dictionary_en_82_765.txt", term_index=0, count_index=1)

# Function to correct spelling using SymSpell
def correct_spelling_symspell(text):
    words = text.split()
    corrected_words = []
    
    for word in words:
        # Get the best correction (or the word itself if no correction found)
        suggestions = sym_spell.lookup(word, Verbosity.TOP, max_edit_distance=2)
        corrected_word = suggestions[0].term if suggestions else word
        corrected_words.append(corrected_word)
    
    return " ".join(corrected_words)

In [18]:
df['review']=df['review'].apply(correct_spelling_symspell)

### Splitting model into train-test/holdout split. Will further split train set into train and validation set later

In [19]:
from sklearn.model_selection import train_test_split
X=df['review']
y=df['sentiment']
X_train, X_holdout, y_train, y_holdout = train_test_split(
    X, y, test_size=0.3, random_state=42,stratify=y)

train_df=pd.concat([X_train,y_train],axis=1)
#X=train_df['review']
#y=train_df['sentiment']
#X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.3, random_state=42,stratify=y)

### Converting text into numbers

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
X_train=tv.fit_transform(X_train)
#X_valid=tv.transform(X_valid)
X_holdout=tv.transform(X_holdout)

In [21]:
X_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3316617 stored elements and shape (35000, 51939)>

### Converting data into tensors 

In [22]:
from torch.utils.data import DataLoader, TensorDataset

X_train=torch.from_numpy(X_train.toarray()).to(torch.float32)
y_train=torch.from_numpy(np.array(y_train)).to(torch.float32)
train_dataset=TensorDataset(X_train,y_train)
train_loader = DataLoader(train_dataset, batch_size=32)


#X_valid=torch.from_numpy(X_valid.toarray()).to(torch.float32)
#y_valid=torch.from_numpy(np.array(y_valid)).to(torch.float32)
#valid_dataset=TensorDataset(X_valid,y_valid)
#valid_loader = DataLoader(valid_dataset, batch_size=32)


X_holdout=torch.from_numpy(X_holdout.toarray()).to(torch.float32)
y_holdout=torch.from_numpy(np.array(y_holdout)).to(torch.float32)
holdout_dataset=TensorDataset(X_holdout,y_holdout)
holdout_loader = DataLoader(holdout_dataset, batch_size=32)

In [23]:
train_dataset

In [24]:
data,label=next(iter(train_loader))
print(f'data shape: {data.shape}')
print(f'data dtype: {data.dtype}')
print(f'label shape: {label.shape}')
print(f'data dtype: {label.dtype}')




data shape: torch.Size([32, 51939])
data dtype: torch.float32
label shape: torch.Size([32])
data dtype: torch.float32


In [25]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'

### LSTM Model architecture

In [26]:
input_size = data.shape[1] # Each features
hidden_dim = 128
output_dim = 1  # Binary classification (positive/negative)

batchsize=32


class model(nn.Module):
    def __init__(self,input_size,hidden_dim,output_dim,num_layers,dropout):
        super(model,self).__init__()
        self.num_layers=num_layers
        self.hidden_dim=hidden_dim
        self.lstm=nn.LSTM(input_size,hidden_dim,num_layers,batch_first=True,dropout=dropout)
        self.fc1 = nn.Linear(hidden_dim, output_dim)
        

    def forward(self,x):
        h0=torch.zeros(self.num_layers,x.size(0),self.hidden_dim).to(device)
        c0=torch.zeros(self.num_layers,x.size(0),self.hidden_dim).to(device)

        out,_=self.lstm(x,(h0,c0))
        out = out[:, -1, :]
        x=self.fc1(out)
        

        return x
        
    

In [27]:

hidden_dim = 128
output_dim = 1  # Binary classification (positive/negative)
num_layers = 2

sequence_length=1
learning_rate=0.001
num_epochs=5
net=model(input_size=data.shape[1],hidden_dim=128,output_dim=1,num_layers=2,dropout=0.2).to(device)


### Train and test Loop with stratifiedKFold cross validation technique 

In [28]:
import torchmetrics
from torchmetrics.classification import BinaryAccuracy
from torch.nn import BCEWithLogitsLoss
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold




skf=StratifiedKFold(n_splits=3)
test_loss=[]
total_acc=[]
for fold, (train_idx, test_idx) in enumerate(skf.split(X_train,y_train)):
    print(f"Fold {fold + 1}")
    print("-------")

    train_loader = DataLoader(
            dataset=train_dataset,
            batch_size=batchsize,
            sampler=torch.utils.data.SubsetRandomSampler(train_idx),
        )
    
    valid_loader = DataLoader(
            dataset=train_dataset,
            batch_size=batchsize,
            sampler=torch.utils.data.SubsetRandomSampler(test_idx),
        )
    
    metric=BinaryAccuracy().to(device)
    bce=BCEWithLogitsLoss()
    optim = torch.optim.Adam(net.parameters(), lr=learning_rate) 
    
    for epoch in tqdm(range(num_epochs)):
        running_loss=0
        #train loop
        
        net.train()
        for batch_size,(X,y) in enumerate(train_loader):
            optim.zero_grad()
            
            X=X.unsqueeze(1).to(device) #add in time sequence
            y=y.to(device)
            
            pred=net(X)
            loss=bce(pred.squeeze(),y)
            running_loss+=loss
            
            loss.backward()
            optim.step()
            
        avg_train_loss=running_loss/len(train_loader)
        
        #print('\nTraining Finished\n')
    
        #validation loop
        net.eval()
        with torch.inference_mode():
            for batch_size,(X,y) in enumerate(valid_loader):
                X=X.unsqueeze(1).to(device)
                y=y.to(device)
                pred=net(X)
                running_loss+=bce(pred.squeeze(),y)
                metric(pred.squeeze(),y)
                
            avg_loss=running_loss/len(valid_loader)
            #total accuracy over all validation batches
            avg_acc=metric.compute()
            test_loss.append(avg_loss.item())
            total_acc.append(avg_acc.item())
            #print(f'Epoch: {epoch+1} | Train Loss : {avg_train_loss} | Valid Loss: {avg_loss} | Valid Accuracy: {avg_acc}')
            
            #Reset metric states after each epoch
            metric.reset()
            
        #print('\nValidation Finished\n')
    print(f'Average Validation loss:{np.mean(test_loss)} | Average Validation Accuracy score:{np.mean(total_acc)} ')
        
    

Fold 1
-------


  0%|          | 0/5 [00:00<?, ?it/s]

Average Validation loss:0.7268621802330018 | Average Validation Accuracy score:0.8742264389991761 
Fold 2
-------


  0%|          | 0/5 [00:00<?, ?it/s]

Average Validation loss:0.4830487594008446 | Average Validation Accuracy score:0.9237850308418274 
Fold 3
-------


  0%|          | 0/5 [00:00<?, ?it/s]

Average Validation loss:0.3411065686494112 | Average Validation Accuracy score:0.9472984830538432 


### Finding best hyperparameters of model using Optuna

In [29]:
import optuna

def define_model(trial):
    params={
    'input_size': data.size(0),
    'hidden_dim' : trial.suggest_int('hidden_dim', 16, 128),
    'num_layers' : trial.suggest_int('num_layers', 2, 4),
    'dropout' : trial.suggest_float("dropout", 0.2, 0.5),
    'output_dim':1
    }
    

    return model(**params)


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(device)
    
    learning_rate = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    metric=BinaryAccuracy().to(device)
    bce=BCEWithLogitsLoss()
    optim = torch.optim.Adam(net.parameters(), lr=learning_rate) 
    
    skf=StratifiedKFold(n_splits=3)
    test_loss=[]
    total_acc=[]
    for fold, (train_idx, test_idx) in enumerate(skf.split(X_train,y_train)):
        print(f"Fold {fold + 1}")
        #print("-------")
    
        train_loader = DataLoader(
                dataset=train_dataset,
                batch_size=batchsize,
                sampler=torch.utils.data.SubsetRandomSampler(train_idx),
            )
        
        valid_loader = DataLoader(
                dataset=train_dataset,
                batch_size=batchsize,
                sampler=torch.utils.data.SubsetRandomSampler(test_idx),
            )
        
        metric=BinaryAccuracy().to(device)
        bce=BCEWithLogitsLoss()
        optim = torch.optim.Adam(net.parameters(), lr=learning_rate) 
        
        for epoch in range(num_epochs):
            running_loss=0
            #train loop
            
            net.train()
            for batch_size,(X,y) in enumerate(train_loader):
                optim.zero_grad()
                
                X=X.unsqueeze(1).to(device) #add in time sequence
                y=y.to(device)
                
                pred=net(X)
                loss=bce(pred.squeeze(),y)
                running_loss+=loss
                
                loss.backward()
                optim.step()
                
            avg_train_loss=running_loss/len(train_loader)
            
            #print('\nTraining Finished\n')
        
            #validation loop
            net.eval()
            with torch.inference_mode():
                for batch_size,(X,y) in enumerate(valid_loader):
                    X=X.unsqueeze(1).to(device)
                    y=y.to(device)
                    pred=net(X)
                    running_loss+=bce(pred.squeeze(),y)
                    metric(pred.squeeze(),y)
                    
                avg_loss=running_loss/len(valid_loader)
                #total accuracy over all validation batches
                avg_acc=metric.compute()
                test_loss.append(avg_loss.item())
                total_acc.append(avg_acc.item())
                print(f'Epoch: {epoch+1} | Train Loss : {avg_train_loss} | Valid Loss: {avg_loss} | Valid Accuracy: {avg_acc}')
                
                #Reset metric states after each epoch
                metric.reset()
                
                acc=np.mean(total_acc)
                trial.report(acc, epoch)
                
                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()
                    
    return np.mean(total_acc)

        

### Taking note of best parameters

In [30]:
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
        print(" {}: {}".format(key, value))

[I 2024-12-22 18:58:28,583] A new study created in memory with name: no-name-b12a0c35-14b8-4b7a-9aae-c3cb34a8e52b


Fold 1
Epoch: 1 | Train Loss : 0.34745606780052185 | Valid Loss: 0.9920234084129333 | Valid Accuracy: 0.8794891834259033
Epoch: 2 | Train Loss : 0.14096730947494507 | Valid Loss: 0.6301586031913757 | Valid Accuracy: 0.875803530216217
Epoch: 3 | Train Loss : 0.08516965806484222 | Valid Loss: 0.6092954874038696 | Valid Accuracy: 0.8716893792152405
Epoch: 4 | Train Loss : 0.05820951610803604 | Valid Loss: 0.6112860441207886 | Valid Accuracy: 0.8770892024040222
Epoch: 5 | Train Loss : 0.02834978513419628 | Valid Loss: 0.5875921249389648 | Valid Accuracy: 0.8717750906944275
Fold 2
Epoch: 1 | Train Loss : 0.20717372000217438 | Valid Loss: 0.49243101477622986 | Valid Accuracy: 0.9819148182868958


/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:497: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(


Epoch: 2 | Train Loss : 0.11645226925611496 | Valid Loss: 0.308785617351532 | Valid Accuracy: 0.9722293615341187


/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:497: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(


...

Epoch: 5 | Train Loss : 7.80563204898499e-05 | Valid Loss: 0.00015618576435372233 | Valid Accuracy: 1.0
Fold 3


/opt/conda/lib/python3.10/site-packages/optuna/trial/_trial.py:497: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(


Epoch: 1 | Train Loss : 4.655452721635811e-05 | Valid Loss: 9.312393376603723e-05 | Valid Accuracy: 1.0
Epoch: 2 | Train Loss : 9.637985385779757e-06 | Valid Loss: 1.928608980961144e-05 | Valid Accuracy: 1.0
Epoch: 3 | Train Loss : 9.791020420379937e-05 | Valid Loss: 0.0001958323991857469 | Valid Accuracy: 1.0
Epoch: 4 | Train Loss : 0.0001418380852555856 | Valid Loss: 0.0002836876374203712 | Valid Accuracy: 1.0


[I 2024-12-22 19:09:30,843] Trial 3 finished with value: 1.0 and parameters: {'hidden_dim': 75, 'num_layers': 2, 'dropout': 0.40426109846789293, 'lr': 0.0008370321623029236}. Best is trial 3 with value: 1.0.


Epoch: 5 | Train Loss : 0.0004855002916883677 | Valid Loss: 0.0009710087324492633 | Valid Accuracy: 1.0
Study statistics: 
  Number of finished trials:  4
  Number of pruned trials:  0
  Number of complete trials:  4
Best trial:
  Value:  1.0
  Params: 
 hidden_dim: 75
 num_layers: 2
 dropout: 0.40426109846789293
 lr: 0.0008370321623029236


### Best Params + Learning Rate, retrain model using this now

In [31]:
best_params={'hidden_dim':29,'num_layers':3,'dropout':0.2941054765241931,'input_size': data.size(1),'output_dim':1}
best_lr=7.579089169356476e-05




In [32]:
net=model(**best_params)
net

model(
  (lstm): LSTM(51939, 29, num_layers=3, batch_first=True, dropout=0.2941054765241931)
  (fc1): Linear(in_features=29, out_features=1, bias=True)
)

In [33]:
import torchmetrics
from torchmetrics.classification import BinaryAccuracy
from torch.nn import BCEWithLogitsLoss
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold




net=model(**best_params).to(device)

skf=StratifiedKFold(n_splits=3)


test_loss=[]
total_acc=[]
for fold, (train_idx, test_idx) in enumerate(skf.split(X_train,y_train)):
    print(f"Fold {fold + 1}")
    print("-------")

    train_loader = DataLoader(
            dataset=train_dataset,
            batch_size=batchsize,
            sampler=torch.utils.data.SubsetRandomSampler(train_idx),
        )
    
    valid_loader = DataLoader(
            dataset=train_dataset,
            batch_size=batchsize,
            sampler=torch.utils.data.SubsetRandomSampler(test_idx),
        )
    
    metric=BinaryAccuracy().to(device)
    bce=BCEWithLogitsLoss()
    optim = torch.optim.Adam(net.parameters(), lr=best_lr) 
    
    for epoch in tqdm(range(30)):
        running_loss=0
        #train loop
        
        net.train()
        for batch_size,(X,y) in enumerate(train_loader):
            optim.zero_grad()
            
            X=X.unsqueeze(1).to(device) #add in time sequence
            y=y.to(device)
            
            pred=net(X)
            loss=bce(pred.squeeze(),y)
            running_loss+=loss
            
            loss.backward()
            optim.step()
            
        avg_train_loss=running_loss/len(train_loader)
        
        #print('\nTraining Finished\n')
    
        #validation loop
        net.eval()
        with torch.inference_mode():
            for batch_size,(X,y) in enumerate(valid_loader):
                X=X.unsqueeze(1).to(device)
                y=y.to(device)
                pred=net(X)
                running_loss+=bce(pred.squeeze(),y)
                metric(pred.squeeze(),y)
                
            avg_loss=running_loss/len(valid_loader)
            #total accuracy over all validation batches
            avg_acc=metric.compute()
            test_loss.append(avg_loss.item())
            total_acc.append(avg_acc.item())
            print(f'Epoch: {epoch+1} | Train Loss : {avg_train_loss} | Valid Loss: {avg_loss} | Valid Accuracy: {avg_acc}')
            
            #Reset metric states after each epoch
            metric.reset()
            
        #print('\nValidation Finished\n')
    print(f'Average Validation loss:{np.mean(test_loss)} | Average Validation Accuracy score:{np.mean(total_acc)} ')
        
        

Fold 1
-------


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 | Train Loss : 0.6933891773223877 | Valid Loss: 2.0798075199127197 | Valid Accuracy: 0.4999571442604065
Epoch: 2 | Train Loss : 0.6904617547988892 | Valid Loss: 2.063283681869507 | Valid Accuracy: 0.7477500438690186
Epoch: 3 | Train Loss : 0.588293194770813 | Valid Loss: 1.6267064809799194 | Valid Accuracy: 0.8469186425209045
Epoch: 4 | Train Loss : 0.3679310083389282 | Valid Loss: 1.0666382312774658 | Valid Accuracy: 0.8691180348396301
Epoch: 5 | Train Loss : 0.29344791173934937 | Valid Loss: 0.8906974196434021 | Valid Accuracy: 0.875889241695404
Epoch: 6 | Train Loss : 0.2582138180732727 | Valid Loss: 0.8098644614219666 | Valid Accuracy: 0.8787177801132202
Epoch: 7 | Train Loss : 0.23689362406730652 | Valid Loss: 0.761503279209137 | Valid Accuracy: 0.8820605278015137
Epoch: 8 | Train Loss : 0.21597953140735626 | Valid Loss: 0.7172077298164368 | Valid Accuracy: 0.884631872177124
Epoch: 9 | Train Loss : 0.19549664855003357 | Valid Loss: 0.6759676933288574 | Valid Accuracy: 0.8

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 | Train Loss : 0.22979481518268585 | Valid Loss: 0.48643046617507935 | Valid Accuracy: 0.9957144260406494
Epoch: 2 | Train Loss : 0.189231738448143 | Valid Loss: 0.4127999246120453 | Valid Accuracy: 0.9952858686447144
Epoch: 3 | Train Loss : 0.16978880763053894 | Valid Loss: 0.3807578384876251 | Valid Accuracy: 0.9949430227279663
Epoch: 4 | Train Loss : 0.15731370449066162 | Valid Loss: 0.35904932022094727 | Valid Accuracy: 0.9947715997695923
Epoch: 5 | Train Loss : 0.14450639486312866 | Valid Loss: 0.33345264196395874 | Valid Accuracy: 0.9940859079360962
Epoch: 6 | Train Loss : 0.13592293858528137 | Valid Loss: 0.3163379430770874 | Valid Accuracy: 0.9934859275817871
Epoch: 7 | Train Loss : 0.1266665905714035 | Valid Loss: 0.29739850759506226 | Valid Accuracy: 0.992971658706665
Epoch: 8 | Train Loss : 0.1175612285733223 | Valid Loss: 0.2785824239253998 | Valid Accuracy: 0.9918573498725891
Epoch: 9 | Train Loss : 0.10901477932929993 | Valid Loss: 0.2618800401687622 | Valid Accu

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 | Train Loss : 0.07694429159164429 | Valid Loss: 0.16223423182964325 | Valid Accuracy: 0.9990571141242981
Epoch: 2 | Train Loss : 0.06390182673931122 | Valid Loss: 0.13704071938991547 | Valid Accuracy: 0.9990571141242981
Epoch: 3 | Train Loss : 0.055110394954681396 | Valid Loss: 0.12000365555286407 | Valid Accuracy: 0.9989713430404663
Epoch: 4 | Train Loss : 0.04924679175019264 | Valid Loss: 0.10834318399429321 | Valid Accuracy: 0.9989713430404663
Epoch: 5 | Train Loss : 0.04730742424726486 | Valid Loss: 0.10462711751461029 | Valid Accuracy: 0.9988856315612793
Epoch: 6 | Train Loss : 0.0409182533621788 | Valid Loss: 0.09180765599012375 | Valid Accuracy: 0.9988856315612793
Epoch: 7 | Train Loss : 0.03792837634682655 | Valid Loss: 0.08618457615375519 | Valid Accuracy: 0.9986284971237183
Epoch: 8 | Train Loss : 0.03644794225692749 | Valid Loss: 0.08317241817712784 | Valid Accuracy: 0.9986284971237183
Epoch: 9 | Train Loss : 0.03440827131271362 | Valid Loss: 0.07948311418294907 | 

### Testing on holdout set

https://www.youtube.com/watch?v=4i4C3ejTdgs

In [34]:
from torchmetrics.classification import BinaryAccuracy
from torchmetrics.classification import BinaryAUROC
auroc = BinaryAUROC().to(device)
metric=BinaryAccuracy().to(device)
for b,(X,y) in enumerate(holdout_loader):
    X=X.unsqueeze(1).to(device)
    y=y.to(device)
    #print(X.shape)
    
    pred=net(X)
    #print(pred.shape)
    acc=metric(pred.squeeze(),y)
    auroc.update(pred.squeeze(), y)
final_acc=metric.compute()
final_auroc=auroc.compute()
print('TEST SET')
print(f'final_acc: {final_acc.item()}')
print(f'final_auroc: {final_auroc.item()}')
    

TEST SET
final_acc: 0.8798666596412659
final_auroc: 0.943869948387146
